#### I am crating this notebook since I realised that I should consolidate all the preprocessing of the data before training

In [1]:
import pandas as pd
import numpy as np
import os
import re
from ast import literal_eval

In [2]:
df = pd.read_csv('/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/data/cards_data.csv')
df.sample(5)

,id,name,cmc,mana_cost,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,rarity,reserved,released_at,booster,usd
11873,6865b8c7-b260-48e8-a905-64cbf06b7e57,"Kaya, Intangible Slayer",7.0,{3}{W}{W}{B}{B},"['B', 'W']",True,['Planeswalker'],['Kaya'],Hexproof\n+2: Each opponent loses 3 life and y...,"['Hexproof', 'Scry']",non-creature,non-creature,False,rare,False,2023-02-03,True,0.35
14924,5e1903c2-6b22-4e2c-99fa-8e259fff29c5,Needle Spires,0.0,{0},['No Colors'],False,['Land'],['none'],Needle Spires enters the battlefield tapped.\n...,['no keywords'],non-creature,non-creature,False,rare,False,2020-09-25,False,0.27
5849,e76c0c83-3e87-474d-bc72-1677eed32cfa,Doom Foretold,4.0,{2}{W}{B},"['B', 'W']",False,['Enchantment'],['none'],"At the beginning of each player's upkeep, that...",['no keywords'],non-creature,non-creature,False,rare,False,2019-10-04,True,0.24
3050,0a707ab3-b9b5-422a-8b0f-e963d3ad6606,"Calix, Guided by Fate",3.0,{1}{G}{W},"['G', 'W']",True,"['Enchantment', 'Creature']","['Human', 'Druid']","Constellation — Whenever Calix, Guided by Fate...",['Constellation'],2,2,False,mythic,False,2023-05-12,True,15.78
5010,efedfd1e-5073-47e2-9137-5f2bf4e436e3,Dauntless River Marshal,2.0,{1}{W},['W'],False,['Creature'],"['Human', 'Soldier']",Dauntless River Marshal gets +1/+1 as long as ...,['no keywords'],2,1,False,uncommon,False,2014-07-18,True,0.04


In [3]:
df.drop(columns=['id', 'mana_cost', 'textless'], inplace=True)

In [4]:
df['sub_types'].fillna("['none']", inplace=True)

In [5]:
df['types'] = df['types'].apply(literal_eval)
df['sub_types'] = df['sub_types'].apply(literal_eval)
df['keywords'] = df['keywords'].apply(literal_eval)

In [6]:
df.drop(df[df['colors']=='[nan]'].index, inplace=True)
df.reset_index(drop=True)
df.head()

,name,cmc,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,rarity,reserved,released_at,booster,usd
0,+2 Mace,2.0,['W'],False,[Artifact],[Equipment],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,[Equip],non-creature,non-creature,common,False,2021-07-23,True,0.02
1,Aarakocra Sneak,4.0,['U'],False,[Creature],"[Bird, Rogue]",Flying\nWhen Aarakocra Sneak enters the battle...,[Flying],1,4,common,False,2022-06-10,True,0.06
2,Abaddon the Despoiler,5.0,"['B', 'R', 'U']",True,[Creature],"[Astartes, Warrior]",Trample\nMark of Chaos Ascendant — During your...,"[Mark of Chaos Ascendant, Trample]",5,5,mythic,False,2022-10-07,False,2.81
3,Abandoned Outpost,0.0,['No Colors'],False,[Land],[none],Abandoned Outpost enters the battlefield tappe...,[no keywords],non-creature,non-creature,common,False,2001-10-01,True,0.14
4,Abandoned Sarcophagus,3.0,['C'],False,[Artifact],[none],You may cast spells that have a cycling abilit...,[no keywords],non-creature,non-creature,rare,False,2020-04-17,False,0.09


In [7]:
df['colors'].replace("['No Colors']", "['N']", inplace=True)
df['colors']=df['colors'].apply(literal_eval)
df.reset_index(drop=True, inplace=True)

In [8]:
df['power'] = df['power'].replace('non-creature', 0)
df['toughness'] = df['toughness'].replace('non-creature', 0)

In [9]:
df.drop(df[df['types'].apply(lambda x: '//' in x)].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [10]:
null_texts_ind = df[df['name'].str.contains('//')][['name','oracle_text']].index
df.drop(null_texts_ind, inplace=True)
df.reset_index(drop=True).head()

,name,cmc,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,rarity,reserved,released_at,booster,usd
0,+2 Mace,2.0,[W],False,[Artifact],[Equipment],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,[Equip],0,0,common,False,2021-07-23,True,0.02
1,Aarakocra Sneak,4.0,[U],False,[Creature],"[Bird, Rogue]",Flying\nWhen Aarakocra Sneak enters the battle...,[Flying],1,4,common,False,2022-06-10,True,0.06
2,Abaddon the Despoiler,5.0,"[B, R, U]",True,[Creature],"[Astartes, Warrior]",Trample\nMark of Chaos Ascendant — During your...,"[Mark of Chaos Ascendant, Trample]",5,5,mythic,False,2022-10-07,False,2.81
3,Abandoned Outpost,0.0,[N],False,[Land],[none],Abandoned Outpost enters the battlefield tappe...,[no keywords],0,0,common,False,2001-10-01,True,0.14
4,Abandoned Sarcophagus,3.0,[C],False,[Artifact],[none],You may cast spells that have a cycling abilit...,[no keywords],0,0,rare,False,2020-04-17,False,0.09


In [11]:
df['oracle_text'].fillna('notext', inplace=True)
df.drop(columns=['name'], inplace=True)

In [12]:
df['power'] = pd.to_numeric(df['power'], errors='coerce')
df['toughness'] = pd.to_numeric(df['toughness'], errors='coerce')
df['power'] = df['power'].fillna(-1).astype(int)
df['toughness'] = df['toughness'].fillna(-1).astype(int)

In [13]:
df['released_at'] = pd.to_datetime(df['released_at'])
reference_date = df['released_at'].min()
df['released_at'] = (df['released_at'] - reference_date).dt.days

In [14]:
df['cmc'] = df['cmc'].astype(int)

In [15]:
def process_oracle_text(oracle_text):

    color_mapping = {'{R}': 'Red ', '{U}': 'Blue ', '{G}': 'Green ', '{B}': 'Black ', '{W}': 'White ',
    '{C}': 'Colorless '}

    for token, color in color_mapping.items():
        oracle_text = oracle_text.replace(token, color)

    oracle_text = re.sub(r'\.', '. ', oracle_text)

    oracle_text = re.sub(r'{(\d+)}', lambda x:f'{x.group(1)} ', oracle_text)

    oracle_text = re.sub(r'{T}', 'Tap', oracle_text)

    oracle_text = re.sub(r'{X}', 'X ', oracle_text)

    oracle_text = re.sub("[\(\[].*?[\)\]]", "", oracle_text)

    oracle_text = oracle_text.replace('\n', ' ')

    return oracle_text

In [16]:
df['oracle_text'] = df['oracle_text'].apply(process_oracle_text)

In [19]:
df.sample(1)

,cmc,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,rarity,reserved,released_at,booster,usd
8045,3,[G],False,[Artifact],[Equipment],"When Fractal Harness enters the battlefield, c...",[Equip],0,0,rare,False,10005,False,1.9


In [17]:
directory_path = '/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/data'
df.to_csv(os.path.join(directory_path, 'clean_cards.csv'), index=False)